In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import sys
import os
from pathlib import Path
from pycocotools.coco import COCO
import pandas as pd
import json
import random
import re
current_dir = Path.cwd()
parent_dir = os.path.abspath(os.path.dirname(current_dir))

sys.path.append(parent_dir)

from question_recuperation.question_utils import questions_recuperation
from image_recuperation.image_utils import calculate_similarity



2024-05-22 19:17:58.615896: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-22 19:17:59.609241: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/jupyter-tfg2324vqa/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import ViltProcessor, ViltForQuestionAnswering

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model_vqa = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")


from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('stabilityai/stablelm-zephyr-3b')
model_text = AutoModelForCausalLM.from_pretrained(
    'stabilityai/stablelm-zephyr-3b',
    device_map="auto"
)



/home/jupyter-tfg2324vqa/.local/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
dataDir='.'
dataType='val2014'
annFile='%s/annotations/captions_%s.json'%(dataDir,dataType)
subtypes=['results', 'totalEval', 'imgEval']
[resFile, evalImgsFile, evalFile]= ['%s/results/captions_%s.json'%(dataDir,subtype) for subtype in subtypes]

In [4]:
coco = COCO('../../COCO/dataset/instances_train2017.json')

loading annotations into memory...
Done (t=12.24s)
creating index...
index created!


In [5]:
num_images_rec = 50 
num_questions_rec = 20
dataset_name = 'dataset_2'
vocab=pd.read_pickle(f'../question_recuperation/embeddings/gte-small_question_embeddings_{dataset_name}.pickle')
df = pd.DataFrame(vocab)

# Caption generation


In [6]:
content_template = """Act as an image caption generator.
Read pairs of questions and answer.
Generate a short description based on the pairs information.
Be brief limiting yourdelf to the information you have.
Here are the question-answer pairs:
{pairs}
Just answer with the generated caption"""

temp = 0.8
n_q = 8


numbers = []

images_path = '../description_evaluation/images/'
test_images_files = [images_path+f+'.jpg' for f in ['girl_dog']]
#existing_images_ids = [file_name[:-4] for file_name in existing_images_files]
image_captions = []
for file in test_images_files:
        print('-'*10)
        print(file[:-4])
        image = Image.open(file)
        df_scores = calculate_similarity(image, 'embeddings', 'cos', coco, dataset_name,'densenet')
        questions_rec = questions_recuperation(df,df_scores,num_images_rec,num_questions_rec)
        print('Questions recovered')

        #vqa
        questions = questions_rec[:n_q]
        answers = {}
        for q in questions:
            encoding = processor(image, q, return_tensors="pt")
            outputs = model_vqa(**encoding)
            logits = outputs.logits
            idx = logits.argmax(-1).item()
            answer = model_vqa.config.id2label[idx]
            answers[q] = answer
            print("Predicted answer:", answer)
        print('Answers predicted')

        #caption
        #format prompt
        questions_answers = ""
        for key in answers:
            questions_answers += "Question: " + key + " Answer: " + answers[key] + "\n" 
        content = content_template.format(pairs = questions_answers[:-1])
        prompt = [{'role': 'user', 'content': content}]
        inputs = tokenizer.apply_chat_template(
            prompt,
            add_generation_prompt=True,
            return_tensors='pt'
        )
        #predict caption
        tokens = model_text.generate(
            inputs.to(model_text.device),
            max_new_tokens=1024,
            temperature=temp,
            do_sample=True
        )

        conversation = tokenizer.decode(tokens[0], skip_special_tokens=False)
        print(conversation)
        print('Caption generated')
        print('-'*10)
        image_captions.append((file[:-4], conversation))
    
with open("./results/captions_2.txt", "a") as file:
    for cap in image_captions:
        file.write(f'{cap[0]}\n')
        file.write(f'{cap[1]}\n')

----------
../description_evaluation/images/girl_dog


/home/jupyter-tfg2324vqa/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jupyter-tfg2324vqa/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/jupyter-tfg2324vqa/project/question_recuperation/question_utils.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

Questions recovered
Predicted answer: brown
Predicted answer: walking
Predicted answer: black
Predicted answer: yes
Predicted answer: backpack
Predicted answer: sidewalk
Predicted answer: no
Predicted answer: black
Answers predicted


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<|user|>
Act as an image caption generator.
Read pairs of questions and answer.
Generate a short description based on the pairs information.
Be brief limiting yourdelf to the information you have.
Here are the question-answer pairs:
Question: What color is the dog? Answer: brown
Question: What is the man doing? Answer: walking
Question: What color is his luggage? Answer: black
Question: Is it sunny? Answer: yes
Question: What does the woman have on her back? Answer: backpack
Question: Where is the dog? Answer: sidewalk
Question: Is the man traveling? Answer: no
Question: What color is the suitcase on the right? Answer: black
Just answer with the generated caption<|endoftext|>
<|assistant|>
A brown dog is walking on a sidewalk with a black suitcase on the right and a black backpack on a woman's back. It is sunny and the man is not traveling.<|endoftext|>
Caption generated
----------
